# Talking to CSV and Excel files with LangChain

In [1]:
!pip -q install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.3 MB/s eta 0:00:00


In [8]:
!pip -q install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_KEY"] = 

In [3]:
!pip show langchain

Name: langchain
Version: 0.2.8
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Dataset - Black Friday Sales


https://www.kaggle.com/datasets/sdolezel/black-friday

In [4]:
!wget -q https://www.dropbox.com/s/8y6a1zloiscuo5d/black_friday_sales.zip
!unzip -q black_friday_sales.zip

In [5]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

In [6]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


## CSV Agent

NOTE: this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.

In [10]:
from langchain_experimental.agents import create_csv_agent
from langchain.llms import OpenAI

In [12]:
agent = create_csv_agent(OpenAI(temperature=0),
                         '/content/train.csv',
                         verbose=True,
                         allow_dangerous_code=True)


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df}

Begin!
Question: {input}
{agent_scratchpad}

In [23]:
agent.run("how many rows are there?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)550068550068 rows is a lot, but I need to keep going
Final Answer: 550068

> Finished chain.


'550068'

In [24]:
agent.run("how many people are female?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows where the Gender column is equal to "F"
Action: python_repl_ast
Action Input: df[df['Gender'] == 'F'].count()User_ID                       135809
Product_ID                    135809
Gender                        135809
Age                           135809
Occupation                    135809
City_Category                 135809
Stay_In_Current_City_Years    135809
Marital_Status                135809
Product_Category_1            135809
Product_Category_2             91530
Product_Category_3             37594
Purchase                      135809
dtype: int64135809 people are female
Final Answer: 135809

> Finished chain.


'135809'

In [27]:
agent.run("how many people have stayed more than 1 year in the city?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the Stay_In_Current_City_Years column is greater than 1.
Action: python_repl_ast
Action Input: df[df['Stay_In_Current_City_Years'] > 1]        User_ID Product_ID Gender    Age  Occupation City_Category  \
0       1000001  P00069042      F   0-17          10             A   
1       1000001  P00248942      F   0-17          10             A   
2       1000001  P00087842      F   0-17          10             A   
3       1000001  P00085442      F   0-17          10             A   
5       1000003  P00193542      M  26-35          15             A   
...         ...        ...    ...    ...         ...           ...   
550054  1006018  P00370293      M  36-45           1             C   
550057  1006023  P00370853      M  26-35           0             C   
550062  1006032  P00372445      M  46-50           7             A   
550064  1006035  P00375436      F  26-35           1      

'197123'

In [29]:
agent.run("how many people have stayed more than 2 years in the city and are female?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe based on the Stay_In_Current_City_Years and Gender columns, and then count the number of rows.
Action: python_repl_ast
Action Input: df[(df['Stay_In_Current_City_Years'] > 2) & (df['Gender'] == 'F')]        User_ID Product_ID Gender    Age  Occupation City_Category  \
70      1000018  P00366542      F  18-25           3             B   
71      1000018  P00190742      F  18-25           3             B   
72      1000018  P00151842      F  18-25           3             B   
73      1000018  P00112642      F  18-25           3             B   
74      1000018  P00118442      F  18-25           3             B   
...         ...        ...    ...    ...         ...           ...   
550035  1005989  P00370853      F   0-17          10             C   
550040  1005997  P00370853      F  26-35           7             A   
550047  1006009  P00372445      F  26-35          12             C   
550051  1006013  P003

'24520'

In [30]:
agent.run("Are there more males or females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females in the dataframe
Action: python_repl_ast
Action Input: df['Gender'].value_counts()Gender
M    348129
F    117213
Name: count, dtype: int64 I now know the final answer
Final Answer: There are more males than females in the dataframe.

> Finished chain.


'There are more males than females in the dataframe.'

## LangChain CSV Loader

In [31]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/train.csv')
data = loader.load()

In [ ]:
#custom chain etc

## Convert XLS file to csv

In [32]:
!pip install openpyxl

In [33]:
!wget -q https://www.dropbox.com/s/co5kwipba9hpumt/excel_files.zip
!unzip -q excel_files.zip

In [34]:
xls_file = r'/content/excel_file_example.xlsx'
output_csv = r'/content/excel_file_example.csv'

# Read the XLS file using pandas and openpyxl as the engine
data = pd.read_excel(xls_file, engine='openpyxl')

# Save the data as a CSV file
data.to_csv(output_csv, index=False)

In [36]:
agent = create_csv_agent(OpenAI(temperature=0),
                         '/content/excel_file_example.csv',
                         verbose=True,
                         allow_dangerous_code=True)

In [37]:
agent.run("What are the column names?")



> Entering new AgentExecutor chain...
Thought: I need to access the column names of the dataframe.
Action: python_repl_ast
Action Input: df.columnsIndex(['0', 'First Name', 'Last Name', 'Gender', 'Country', 'Age', 'Date',
       'Id'],
      dtype='object') I now know the column names.
Final Answer: Index(['0', 'First Name', 'Last Name', 'Gender', 'Country', 'Age', 'Date',
       'Id'],
      dtype='object')

> Finished chain.


"Index(['0', 'First Name', 'Last Name', 'Gender', 'Country', 'Age', 'Date',\n       'Id'],\n      dtype='object')"

In [38]:
agent.run("What is the average age?")



> Entering new AgentExecutor chain...
Thought: I need to find the average of the 'Age' column
Action: python_repl_ast
Action Input: df['Age'].mean()33.26I now know the final answer
Final Answer: The average age is 33.26.

> Finished chain.


'The average age is 33.26.'

In [39]:
agent.run("Which country appears the most and how many times does it appear?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of occurrences of each country and then find the maximum value.
Action: python_repl_ast
Action Input: df['Country'].value_counts()Country
United States    48
Great Britain    28
France           24
Name: count, dtype: int64I now know the maximum value is 48.
Action: python_repl_ast
Action Input: df['Country'].value_counts().max()48I now know the final answer is United States with 48 occurrences. 
Final Answer: United States with 48 occurrences.

> Finished chain.


'United States with 48 occurrences.'

In [40]:
agent.run("What is the ratio of males to females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females and then divide the male count by the female count.
Action: python_repl_ast
Action Input: df['Gender'].value_counts()Gender
Female    76
Male      24
Name: count, dtype: int6476 females and 24 males
Action: python_repl_ast
Action Input: 24/760.31578947368421050.3157894736842105 is the ratio of males to females
Final Answer: The ratio of males to females is 0.3157894736842105.

> Finished chain.


'The ratio of males to females is 0.3157894736842105.'